# DFL benchmark - training
This is a simple benchmark script for DFL.  
It classifies each frame image in the video into 4 classes（'background','challenge','play','throwin'） 
It does not use temporal information, so it may not be competitive on its own for this competition, but it could be used as a feature extractor for more advanced models.

In [1]:
!nvidia-smi

Failed to initialize NVML: Unknown Error


In [2]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
from IPython.display import Video
import cv2

# setting

In [3]:
DEBUG = False

In [4]:
class CFG:
    train_csv = "/workdir/work/input/train.csv"
    result_dir = f"/workdir/work/output/train_images_flow/"
    arround_time = 1
    video_dir = "/workdir/work/input/train"

    # optical flow settings
    # params for ShiTomasi corner detection
    feature_params = dict( maxCorners = 100,
                        qualityLevel = 0.3,
                        minDistance = 7,
                        blockSize = 7 )

    # Parameters for lucas kanade optical flow
    lk_params = dict( winSize  = (15,15),
                    maxLevel = 2,
                    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    
    crop_range = 600
    area_thr = 0.1
    IMG_SIZE = 456

In [5]:
def get_maxflow_area(x, y, width, height, crop_range=CFG.crop_range):
    crop_x_min = int(x) - (crop_range/2)
    crop_x_max = int(x) + (crop_range/2)
    crop_x_min = max(0, crop_x_min)
    crop_x_max = min(width, crop_x_max)

    crop_y_min = int(y) - (crop_range/2)
    crop_y_max = int(y) + (crop_range/2)
    crop_y_min = max(0, crop_y_min)
    crop_y_max = min(height, crop_y_max)

    return int(crop_x_min), int(crop_x_max), int(crop_y_min), int(crop_y_max)

In [6]:
def get_green_area(img):
    # HSV
    img_HSV = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

    # creat green mask (H value for green is from 100/360*179 until 180/360*179) in OpenCV
    lower_green = np.array([100 / 360 * 179, 0, 0])
    upper_green = np.array([180 / 360 * 179, 255, 255])
    green_mask = cv2.inRange(img_HSV, lower_green, upper_green)

    # crop green area
    img_green_masked = cv2.bitwise_and(img, img, mask=green_mask)
    img_green_masked = cv2.cvtColor(img_green_masked, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, hierarchy = cv2.findContours(img_green_masked, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if not contours:
        return int(img.shape[1]*0.5), int(img.shape[0]*0.5), img.shape[1], img.shape[0]
    # Find the contour with the maximum area.
    c = max(contours, key=cv2.contourArea)

    # Get bounding rectangle
    x, y, w, h = cv2.boundingRect(c)

    return x, y, w, h

In [7]:
def crop_maxflow_area(cap_, fps_, frame_num_, event_):
    """色相が緑っぽいエリアをフィールドとしてクロップして、optical flowを計算。1secでflowが最も大きい点を中心としてcropする。
    
    """
    cap_.set(cv2.CAP_PROP_POS_FRAMES, frame_num_)
    successed, base_frame = cap_.read()
    if not successed:
        return [], 0

    # optical flow line color
    line_color = [255, 100, 0]
    # base frameでフィールドの範囲を決める。1secで大きくカメラ方向が移動するときはだめかも
    x_field, y_field, w_field, h_field = get_green_area(base_frame)
    base_frame = base_frame[y_field:y_field+h_field, x_field:x_field+w_field, :]

    # Take first frame and find corners in it
    base_gray = cv2.cvtColor(base_frame, cv2.COLOR_BGR2GRAY)
    base_point = cv2.goodFeaturesToTrack(base_gray, mask = None, **CFG.feature_params)
    # Create a mask image for drawing purposes
    mask = np.zeros_like(base_frame)

    # draw the tracks
    max_distance = 0
    img_max_dist = base_frame

    if event_ == "start":
        frame_num = frame_num_ - fps_
        cap_.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
    elif event_ == "end":
        frame_num = frame_num_
        cap_.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
    else:# play系は0.5秒前から
        frame_num = frame_num_ - fps_*0.5
        cap_.set(cv2.CAP_PROP_POS_FRAMES, frame_num)

    # 1secで基準画像とのoptical flowが最も大きい点を残す
    for i in range(int(fps_)):
        successed, relative_frame = cap_.read()
        if not successed:
            continue
        frame_num += 1

        # base frameで決めたフィールドの範囲をcrop
        relative_frame = relative_frame[y_field : y_field+h_field, x_field : x_field+w_field, :]
        relative_gray = cv2.cvtColor(relative_frame, cv2.COLOR_BGR2GRAY)

        # calculate optical flow
        relative_point, st, err = cv2.calcOpticalFlowPyrLK(base_gray, relative_gray, base_point, None, **CFG.lk_params)

        # Select good points
        good_new = relative_point[st==1]
        good_old = base_point[st==1]

        x_max_distance = int(x_field + (w_field*0.5))
        y_max_distance = int(y_field + (h_field*0.5))
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            x_after, y_after = new.ravel()
            x_before, y_before = old.ravel()
            distance = np.sqrt( (x_after - x_before)**2 + (y_after - y_before)**2 )

            # distanceが大きい、かつ、検出点が画像の外側area_thr%にないときに移動距離が最大と判定する
            if distance > max_distance and (w_field*CFG.area_thr < x_after < w_field*(1.0 -CFG.area_thr)) and (h_field*CFG.area_thr < y_after < y_field*(1.0 - CFG.area_thr)):
                max_distance = distance
                x_max_distance = x_after
                y_max_distance = y_after
                # 画像にoptical flow の線を追加する(本当はtop5だけ描くとかの方がいいかも？)
                mask = cv2.line(mask, (int(x_after),int(y_after)), (int(x_before), int(y_before)), line_color, 2)
                relative_frame = cv2.line(relative_frame, (int(x_after),int(y_after)), (int(x_before), int(y_before)), line_color, 2)
                img_max_dist = cv2.circle(relative_frame, (int(x_after),int(y_after)), 5, line_color, -1)

    # optical flowの最も大きい点を中心にクロップ(サイズが足りないときは横方向だけとか縦方向だけの場合もあり)
    x_min, x_max, y_min, y_max = get_maxflow_area(int(x_max_distance), int(y_max_distance), w_field, h_field)
    croped_flow_image = img_max_dist[y_min:y_max, x_min:x_max, :]
    croped_flow_image = cv2.resize(croped_flow_image, dsize=(CFG.IMG_SIZE, CFG.IMG_SIZE))

    return croped_flow_image, max_distance


In [8]:
# save images around event in result dir
if not os.path.exists(CFG.result_dir):
    os.makedirs(CFG.result_dir)

In [9]:
df = pd.read_csv(CFG.train_csv)
df = df[["video_id", "time", "event"]]
display(df)
# video_ids = df["video_id"].unique()#全動画を対象にする場合はこれ
video_ids = ["1606b0e6_0", "1606b0e6_1", "cfbe2e94_0", "cfbe2e94_1"]
if DEBUG:
    video_ids = ["1606b0e6_0", "cfbe2e94_0"]
print(video_ids)

,video_id,time,event
0,1606b0e6_0,200.265822,start
1,1606b0e6_0,201.150000,challenge
2,1606b0e6_0,202.765822,end
3,1606b0e6_0,210.124111,start
4,1606b0e6_0,210.870000,challenge
...,...,...,...
11213,ecf251d4_0,3056.587000,challenge
11214,ecf251d4_0,3058.072895,end
11215,ecf251d4_0,3068.280519,start
11216,ecf251d4_0,3069.547000,throwin


['1606b0e6_0', '1606b0e6_1', 'cfbe2e94_0', 'cfbe2e94_1']


In [10]:
def make_saved_event_df(save_event_df_, video_id_, frame_, target_, distance_):
    save_list = [video_id_, frame_, target_, distance_]
    if len(save_event_df_) == 0:
        save_event_df_ = pd.DataFrame([save_list], columns=["video_id", "frame", "event", "distance"])
    else:
        tmp_event_df_ = pd.DataFrame([save_list], columns=["video_id", "frame", "event", "distance"])
        save_event_df_ = pd.concat([save_event_df_, tmp_event_df_])
    return save_event_df_

In [11]:
def extract_training_images(video_id):
    saved_image_df = pd.DataFrame()
    video_path = f"{CFG.video_dir}/{video_id}.mp4"
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frame = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    if not cap.isOpened():
        print(f"video {video_path} cannot be opened.")
        return
    
    df_video = df[df.video_id == video_id]
    if DEBUG:
        df_video = df_video.head(10)
    print(video_id, df_video.shape)

    #crr_statu => background, play, challenge, throwin
    time_and_event = df_video[['time','event']].values
    idx = 0
    saved_frame_list = []
    save_event_df = pd.DataFrame()
    current_frame = 0
    start_time = time.time()
    elapsed_time_sum = 0
    while idx < len(time_and_event):
        one_loop_start_time = time.time()
        if DEBUG and idx > 3:
            break
        current_time = time_and_event[idx, 0]
        current_event = time_and_event[idx, 1]
        current_frame = int(current_time*fps)

        if current_event == 'start':
            target = 'background'
            croped_flow_image, max_distance = crop_maxflow_area(cap, fps, current_frame, current_event)
            if max_distance != 0:
                out_file = f'{CFG.result_dir}/{video_id}_{current_frame:06}.jpg'
                cv2.imwrite(out_file, croped_flow_image)
                save_event_df = make_saved_event_df(save_event_df, video_id, current_frame, target, max_distance)
            idx += 1
        
        #endはsaveせずにやってみる
        elif current_event == 'end':
            idx += 1
            continue
            # target = 'background'
            # croped_flow_image, max_distance = crop_maxflow_area(cap, fps, current_frame, current_event)
            # if max_distance != 0:
            #     out_file = f'{CFG.result_dir}/{video_id}_{current_frame:06}.jpg'
            #     cv2.imwrite(out_file, croped_flow_image)
            #     save_event_df = make_saved_event_df(save_event_df, video_id, current_frame, target, max_distance)

        else:
            target = current_event
            croped_flow_image, max_distance = crop_maxflow_area(cap, fps, current_frame, current_event)
            if max_distance != 0:
                out_file = f'{CFG.result_dir}/{video_id}_{current_frame:06}.jpg'
                cv2.imwrite(out_file, croped_flow_image)
                save_event_df = make_saved_event_df(save_event_df, video_id, current_frame, target, max_distance)
            idx += 1
        elapsed_time = time.time() - one_loop_start_time
        elapsed_time_sum += elapsed_time
        elapsed_time_mean = elapsed_time_sum/idx
        print("\r" + f"{int(current_frame)} / {int(total_frame)}, elapsed_time={elapsed_time_mean:04f} sec.", end="")    
    
    elapsed_time = time.time() - start_time
    print(f"  video {video_id} is finished. elapsed time={(elapsed_time/60):04f} min. saved image num={len(save_event_df)}.")
    save_event_df["time"] = save_event_df["frame"]/fps
    return save_event_df

In [12]:
train_images_df = pd.DataFrame()
for video_id in video_ids:            
    video_event_df = extract_training_images(video_id)
    if len(train_images_df) == 0:
        train_images_df = video_event_df
    else:
        train_images_df = pd.concat([train_images_df, video_event_df])
print('done')

1606b0e6_0 (1000, 3)
75330 / 85915, elapsed_time=0.174660 sec.  video 1606b0e6_0 is finished. elapsed time=2.908842 min. saved image num=359.
1606b0e6_1 (1249, 3)
84662 / 85138, elapsed_time=0.180230 sec.  video 1606b0e6_1 is finished. elapsed time=3.749716 min. saved image num=385.
cfbe2e94_0 (823, 3)
80391 / 90500, elapsed_time=0.170834 sec.  video cfbe2e94_0 is finished. elapsed time=2.340875 min. saved image num=560.
cfbe2e94_1 (763, 3)
89358 / 89925, elapsed_time=0.168430 sec.  video cfbe2e94_1 is finished. elapsed time=2.139463 min. saved image num=518.
done


In [13]:
train_images_df.to_csv("/workdir/work/output/saved_train_flowimages.csv", index=False)
print("Completed.")

Completed.


In [14]:
len(train_images_df)

1822

In [15]:
train_images_df["event"].value_counts

<bound method IndexOpsMixin.value_counts of 0    background
0    background
0          play
0    background
0          play
        ...    
0          play
0    background
0          play
0    background
0       throwin
Name: event, Length: 1822, dtype: object>

In [16]:
train_images_df["event"].value_counts()

play          831
background    808
challenge     125
throwin        58
Name: event, dtype: int64